In [1]:
# Importing PyAudio to receive user voice
import pyaudio
import wave

In [2]:
CHUNK = 1024
FORMAT = pyaudio.paInt16
CHANNELS = 2
RATE = 44100
RECORD_SECONDS = 5
WAVE_OUTPUT_FILENAME = "output.wav"

In [3]:
# Recording voice

p = pyaudio.PyAudio()
stream = p.open(format=FORMAT,
                channels=CHANNELS,
                rate=RATE,
                input=True,
                frames_per_buffer=CHUNK)
print("* recording started.  You have 5 seconds to record")
frames = []
for i in range(0, int(RATE / CHUNK * RECORD_SECONDS)):
    data = stream.read(CHUNK)
    frames.append(data)

print("* Successfully recording is completed")


stream.stop_stream()
stream.close()
p.terminate()

* recording started.  You have 5 seconds to record
* Successfully recording is completed


In [4]:
# storing file

wf = wave.open(WAVE_OUTPUT_FILENAME, 'wb')
wf.setnchannels(CHANNELS)
wf.setsampwidth(p.get_sample_size(FORMAT))
wf.setframerate(RATE)
wf.writeframes(b''.join(frames))
wf.close()

In [6]:
import IPython.display

# Librosa for audio
import librosa
# And the display module for visualization
import librosa.display


y, sr = librosa.load('output.wav')
# Play it back!
IPython.display.Audio(data=y, rate=sr)

In [7]:
import ast
import numpy as np

M = librosa.feature.melspectrogram(y=y, sr=sr)
# MFCC = librosa.feature.mfcc(y=y, sr=sr)
mfcc = []
mean=[]
MF = librosa.feature.mfcc(y=y, sr=sr, n_mfcc=13)
for i in MF:
    mfcc.append(list(i))

# m = np.mean(mfcc[0])
for j in mfcc:
    m = np.mean(j)
    mean.append(m)

In [11]:
import numpy as np
mean = np.asarray(mean, dtype=np.float32)
mean = mean.reshape(1,-1)

In [12]:
# load the model from disk and test accuracy

import pickle
filename = 'model_knn_mean.sav'

loaded_model = pickle.load(open(filename, 'rb'))
# print(loaded_model)
result = loaded_model.predict(mean)
result = result[0]
# print(result)

# n_class =['sad', 'happy','angry','fear']
# n_class[result]

ValueError: query data dimension must match training data dimension

---
# Keras Saved

In [96]:
fileName = 'keras_model_ann_mean_colab_72_22.h5'

In [97]:
from keras.models import load_model
from keras.initializers import glorot_uniform
from keras.utils import CustomObjectScope

with CustomObjectScope({'GlorotUniform': glorot_uniform()}):
        pk = load_model(fileName)
# pk = load_model(fileName)

# saving same file in json format
json_string = pk.to_json()

In [98]:
# model reconstruction from JSON:
from keras.models import model_from_json
model = model_from_json(json_string)

In [99]:
res = model.predict(mean)
result = res[0]*100
print(result)
print("Sad: ",result[0])
print("Happy: ", result[1])
print("Angry: ", result[2])
print("Fear: ", result[3])

n_class =['sad', 'happy','angry','fear']
# n_class[result[0]]

[5.5942664e-06 5.6722009e-04 9.3973007e+01 6.0264187e+00]
Sad:  5.5942664e-06
Happy:  0.0005672201
Angry:  93.97301
Fear:  6.0264187
